In [449]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import xgboost as xgb
%matplotlib inline

In [450]:
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
merged = pd.concat([data, test], axis=0)
merged

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [451]:
## stuff on NA columns

# LotFrontage: Linear feet of street connected to property. NA=0?
# Alley: NA = no alley
# MasVnrType: NA = no masonry veneer
# MasVnrArea: NA = 0
# BsmtQual: NA = no basement
# BsmtCond: NA = no basement
# BsmtExposure: NA = no basement
# BsmtFinType1: NA = no basement
# BsmtFinType2: NA = no basement
# Electrical: not clear, we may remove the observation
# FireplaceQu: NA = no fireplace
# Garage(X): NA = no garage
# PoolQC: NA = no pool
# Fence: NA = no fence
# MiscFeature: NA = no misc feature
# GarageYrBlt: NA = no garage

def handle_na(df):
    use_na = [
        'Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure',
        'BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType',
        'GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature'
    ]
    use_0 = [
        'LotFrontage','MasVnrArea','GarageCars','GarageArea'
    ]
    for key in use_na:
        df[key] = df[key].fillna('NA')
    for key in use_0:
        df[key] = df[key].fillna(0)
    
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(min(df['GarageYrBlt']))
    return df

In [452]:
# Basic feature engineering / processing tasks

def feature_engineering(df):
    # drop Id
    df = df.drop(['Id'], axis=1)

    # combine month sold and year sold into one column
    df['SaleDate'] = df['MoSold']/12 + df['YrSold']
    df = df.drop(['MoSold', 'YrSold'], axis=1)

    # convert mssubclass to categorical
    df['MSSubClass'] = df['MSSubClass'].astype(str)

    # for everything that's not a numerical column, convert to categorical
    non_numerical_cols = df.select_dtypes(exclude=np.number).columns
    df[non_numerical_cols] = df[non_numerical_cols].astype('category')
    return df

In [453]:
merged = feature_engineering(handle_na(merged))
merged

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,SalePrice,SaleDate
0,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,WD,Normal,208500.0,2008.166667
1,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,WD,Normal,181500.0,2007.416667
2,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,WD,Normal,223500.0,2008.750000
3,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,0,NA,NA,NA,0,WD,Abnorml,140000.0,2006.166667
4,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,WD,Normal,250000.0,2009.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,WD,Normal,NaN,2006.500000
1455,160,RM,21.0,1894,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,WD,Abnorml,NaN,2006.333333
1456,20,RL,160.0,20000,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,WD,Abnorml,NaN,2006.750000
1457,85,RL,62.0,10441,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,MnPrv,Shed,700,WD,Normal,NaN,2006.583333


In [454]:
# import seaborn as sns
# # Set the plot style
# sns.set_theme(style='whitegrid')

# # Select numerical columns and plot histograms
# numerical_cols = data.select_dtypes(include=np.number).columns
# data[numerical_cols].hist(figsize=(12, 8), bins=20)

# # Add labels and titles
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.title('Histograms of Numerical Features')

# # Show the plot
# plt.show()

In [455]:
# Get the categorical columns
categorical_cols = merged.select_dtypes(include='category').columns

# Perform one-hot encoding
merged = pd.get_dummies(merged, columns=categorical_cols, drop_first=True)

data = merged.iloc[:len(data)]

In [456]:
# Split into train and validation
train, val = train_test_split(data, test_size=0.2, random_state=420)

In [457]:
### Baseline RMSE from just predicting mean
np.sqrt(mean_squared_error(val['SalePrice'], np.mean(val['SalePrice']) * np.ones(val['SalePrice'].shape)))

78056.97104591088

In [458]:
def eval_model(train_pred, val_pred):
    train_mse = mean_squared_error(train['SalePrice'], train_pred)
    print('Train rMSE:', np.sqrt(train_mse))

    val_mse = mean_squared_error(val['SalePrice'], val_pred)
    print('Val rMSE:', np.sqrt(val_mse))

def ensemble(train_preds, y_true=train['SalePrice']):
    # Stack predictions as features for the meta-model
    X_stack = np.column_stack(train_preds)
    meta_model = LinearRegression()
    meta_model.fit(X_stack, y_true)

    # The weights
    weights = meta_model.coef_
    print("Weights for DR Stacking:", weights)

    # MSE CSL
    return meta_model

In [459]:
## Linear model
lm = LinearRegression()
lm.fit(train.drop('SalePrice', axis=1), train['SalePrice'])

train_pred_lm = lm.predict(train.drop('SalePrice', axis=1))
val_pred_lm = lm.predict(val.drop('SalePrice', axis=1))
eval_model(train_pred_lm, val_pred_lm)

### Random Forest (no hyperparameter tuning)
rf = RandomForestRegressor()
rf.fit(train.drop('SalePrice', axis=1), train['SalePrice'])

train_pred_rf = rf.predict(train.drop('SalePrice', axis=1))
val_pred_rf = rf.predict(val.drop('SalePrice', axis=1))
eval_model(train_pred_rf, val_pred_rf)

### Gradient boosting
dtrain = xgb.DMatrix(train.drop('SalePrice', axis=1), label=train['SalePrice'])
dval = xgb.DMatrix(val.drop('SalePrice', axis=1), label=val['SalePrice'])
params = {
        'booster': 'gbtree',
        'subsample': 1.0,
        'colsample_bytree': 0.9,
        'num_parallel_tree': 1,
        'max_depth': 7,
        'min_child_weight': 7,
        'eta': 0.1,
        'gamma': 0.05,
        # 'lambda': 2,
        'random_state': 195,
    }
# Specify validations set to watch performance
results = {}
watchlist = [(dtrain, 'train'), (dval, 'eval')]
bst = xgb.train(params=params, dtrain=dtrain, num_boost_round=100, evals=watchlist, evals_result = results, early_stopping_rounds=10, verbose_eval=False)
# Assuming dtest is already defined and is the test DMatrix
train_pred_gb = bst.predict(dtrain)
val_pred_gb = bst.predict(dval)
eval_model(train_pred_gb, val_pred_gb)

### Stacking
train_preds = [train_pred_rf, train_pred_gb]
val_preds = [val_pred_rf, val_pred_gb]
ensemble_model = ensemble(train_preds)
train_pred_stack, val_pred_stack = ensemble_model.predict(np.column_stack(train_preds)), ensemble_model.predict(np.column_stack(val_preds))
eval_model(train_pred_stack, val_pred_stack)

Train rMSE: 20486.873455038494
Val rMSE: 43455.002144091995
Train rMSE: 12139.065917127942
Val rMSE: 26016.8341460804
Train rMSE: 7060.521406706416
Val rMSE: 25189.450653450924
Weights for DR Stacking: [-0.08939773  1.10034329]
Train rMSE: 6923.405080457644
Val rMSE: 25358.709429052004


Tuning the RF and GB

In [460]:
# ### perform k=5 fold cross validation on random forest, doing grid search over hyperparameters
# from sklearn.model_selection import GridSearchCV
# rf_param_grid = {
#     'max_depth': [5, 10, 20],
#     'min_samples_split': [2, 5, 10, 20, 50],
#     'min_samples_leaf': [2, 5, 10, 20, 50],
# }
# rf = RandomForestRegressor(n_estimators=50, random_state=0)
# grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=5, n_jobs=-1, verbose=2)
# grid_search.fit(data.drop('SalePrice', axis=1), data['SalePrice'])
# print(grid_search.best_params_)

In [461]:
rf_params = {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10} ### seems like default rf does better
rf = RandomForestRegressor(n_estimators=50, random_state=0)
rf.fit(train.drop('SalePrice', axis=1), train['SalePrice'])
train_pred_rf = rf.predict(train.drop('SalePrice', axis=1))
val_pred_rf = rf.predict(val.drop('SalePrice', axis=1))
eval_model(train_pred_rf, val_pred_rf)

Train rMSE: 11582.197316984251
Val rMSE: 25302.286006432412


In [462]:
# ### perform k=5 fold cross validation on xgboost, doing grid search over hyperparameters
# ddata = xgb.DMatrix(data.drop('SalePrice', axis=1), label=data['SalePrice'])

# gb_param_grid = {
#     'max_depth': [3, 5, 7, 10],
#     'eta': [0.01, 0.015, 0.025, 0.05, 0.1], 
#     'gamma': [0.05, 0.3, 0.5, 0.7, 1.0],
#     'subsample': [0.5, 0.7, 0.9, 1.0],
#     'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
#     'min_child_weight': [1, 3, 5, 7]
# }


# ### First optimize eta, gamma, min_child_weight
# params = {
#     'booster': 'gbtree',
#     'num_parallel_tree': 1,
#     'random_state': 0,
# }

# from itertools import product

# best_params = None
# min_mse = float('inf')

# i = 0
# for eta, gamma, min_child_weight in product(gb_param_grid['eta'], gb_param_grid['gamma'], gb_param_grid['min_child_weight']):
#     if i % 10 == 0:
#         print(i)
#         print(min_mse)

#     params['eta'] = eta
#     params['gamma'] = gamma
#     params['min_child_weight'] = min_child_weight

#     res = xgb.cv(
#         params,
#         ddata,
#         num_boost_round=100,
#         nfold=5,
#         seed=0,
#     )
#     mse = res["test-rmse-mean"].values[-1]

#     if mse < min_mse:
#         min_mse = mse
#         best_params = {'eta':eta, 'gamma': gamma, 'min_child_weight': min_child_weight}
#     i += 1

# print("Best Params:", best_params)


# ### Next optimize max_depth, subsample, colsample_bytree
# params = {
#     'booster': 'gbtree',
#     'num_parallel_tree': 1,
#     'random_state': 0,
#     'eta': 0.1,
#     'gamma': 0.05,
#     'min_child_weight': 3
# }

# best_params = None
# min_mse = float('inf')

# i = 0

# for max_depth, subsample, colsample in product(gb_param_grid['max_depth'], gb_param_grid['subsample'], gb_param_grid['colsample_bytree']):
#     if i % 10 == 0:
#         print(i)
#         print(min_mse)

#     params['max_depth'] = max_depth
#     params['subsample'] = subsample
#     params['colsample_bytree'] = colsample

#     res = xgb.cv(
#         params,
#         ddata,
#         num_boost_round=100,
#         nfold=5,
#         seed=0,
#     )
#     mse = res["test-rmse-mean"].values[-1]

#     if mse < min_mse:
#         min_mse = mse
#         best_params = {'max_depth':max_depth, 'subsample': subsample, 'colsample_bytree': colsample}
#     i += 1

# print("Best Params:", best_params)

In [463]:
### final GB model
params = {
    'booster': 'gbtree',
    'num_parallel_tree': 1,
    'random_state': 0,
    'eta': 0.1,
    'gamma': 0.05,
    'min_child_weight': 3,
    'max_depth': 7,
    'subsample': 0.9,
    'colsample_bytree': 0.9
}
# Specify validations set to watch performance
results = {}
watchlist = [(dtrain, 'train'), (dval, 'eval')]
gb = xgb.train(params=params, dtrain=dtrain, num_boost_round=100, evals=watchlist, evals_result = results, early_stopping_rounds=10, verbose_eval=False)
# Assuming dtest is already defined and is the test DMatrix
train_pred_gb = gb.predict(dtrain)
val_pred_gb = gb.predict(dval)
eval_model(train_pred_gb, val_pred_gb)

Train rMSE: 4303.581649372352
Val rMSE: 24863.75561522188


In [464]:
### Stacking
train_preds = [train_pred_rf, train_pred_gb]
val_preds = [val_pred_rf, val_pred_gb]
ensemble_model = ensemble(train_preds)
train_pred_stack, val_pred_stack = ensemble_model.predict(np.column_stack(train_preds)), ensemble_model.predict(np.column_stack(val_preds))
eval_model(train_pred_stack, val_pred_stack)

Weights for DR Stacking: [-0.08175548  1.08539957]
Train rMSE: 4192.996572235531
Val rMSE: 25029.100881187183


In [465]:
### Stacking on test set

# get train set predictions
ddata = xgb.DMatrix(data.drop('SalePrice', axis=1), label=data['SalePrice'])
train_preds = [rf.predict(data.drop('SalePrice', axis=1)), gb.predict(ddata)]
ensemble_model = ensemble(train_preds, y_true=data['SalePrice'])

# run stacking on test set
ids = test['Id']
test = merged.iloc[len(data):].drop(columns=['SalePrice'])
dtest = xgb.DMatrix(test, label=None)
test_pred_rf = rf.predict(test)
test_pred_gb = gb.predict(dtest)
test_preds = [test_pred_rf, test_pred_gb]
final_preds = ensemble_model.predict(np.column_stack(test_preds))

Weights for DR Stacking: [0.07704057 0.93844512]


In [466]:
output = pd.DataFrame({'Id': ids, 'SalePrice': final_preds.squeeze()}) 
output.to_csv('submission.csv', index=False)
output

,Id,SalePrice
0,1461,125357.094365
1,1462,160658.448996
2,1463,176051.618987
3,1464,190493.955979
4,1465,185774.519337
...,...,...
1454,2915,79923.022699
1455,2916,76826.064746
1456,2917,153585.823351
1457,2918,108538.972353


In [467]:
# #### Get the feature importances
# importances = pd.Series(rf.feature_importances_, index=train.drop('SalePrice', axis=1).columns)

# # Sort the feature importances in descending order
# importances_sorted = importances.sort_values(ascending=False)

# # Print the sorted feature importances
# print(importances_sorted.head(20))

# # Select the top 20 features
# top_20_features = importances_sorted.head(20).index

# # Restrict the dataset to only the top 20 features
# train_top_20 = train[top_20_features]
# val_top_20 = val[top_20_features]

# # Create new instances of the LinearRegression and RandomForestRegressor models
# lm_top_20 = LinearRegression()
# rf_top_20 = RandomForestRegressor(n_estimators=5000, max_depth=20)

# # Fit the models to the training data with the top 20 features
# lm_top_20.fit(train_top_20, train['SalePrice'])
# rf_top_20.fit(train_top_20, train['SalePrice'])

# # Predict on the training data with the top 20 features
# train_pred_lm_top_20 = lm_top_20.predict(train_top_20)
# train_pred_rf_top_20 = rf_top_20.predict(train_top_20)

# # Compute the train RMSE for linear regression and random forest with the top 20 features
# train_rmse_lm_top_20 = np.sqrt(mean_squared_error(train['SalePrice'], train_pred_lm_top_20))
# train_rmse_rf_top_20 = np.sqrt(mean_squared_error(train['SalePrice'], train_pred_rf_top_20))

# # Predict on the validation data with the top 20 features
# val_pred_lm_top_20 = lm_top_20.predict(val_top_20)
# val_pred_rf_top_20 = rf_top_20.predict(val_top_20)

# # Compute the validation RMSE for linear regression and random forest with the top 20 features
# val_rmse_lm_top_20 = np.sqrt(mean_squared_error(val['SalePrice'], val_pred_lm_top_20))
# val_rmse_rf_top_20 = np.sqrt(mean_squared_error(val['SalePrice'], val_pred_rf_top_20))

# # Print the train and validation RMSE for linear regression and random forest with the top 20 features
# print('Linear Regression Train RMSE (Top 20 features):', train_rmse_lm_top_20)
# print('Random Forest Train RMSE (Top 20 features):', train_rmse_rf_top_20)
# print('Linear Regression Validation RMSE (Top 20 features):', val_rmse_lm_top_20)
# print('Random Forest Validation RMSE (Top 20 features):', val_rmse_rf_top_20)
